# Switzerland Network Analysis (Unified Pipeline)

Uses `src.analysis` + **Central Config** to calculate metrics.  
Implements **Smart Caching**: Increasing `NUM_RANDOM_SIMULATIONS` in `config.py` triggers re-runs.

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
import pickle
import networkx as nx
from pathlib import Path
from IPython.display import display

project_root = Path("../../").resolve()
sys.path.append(str(project_root))

from src.analysis.metrics import NetworkAnalyzer
from src.analysis.storage import ResultsManager
from src.analysis.visualizer import NetworkVisualizer
from src.analysis.config import AnalysisConfig

In [ ]:
COUNTRY = "switzerland"
GRAPH_PATH = project_root / AnalysisConfig.get_graph_path(COUNTRY)

NUM_SIMULATIONS = AnalysisConfig.NUM_RANDOM_SIMULATIONS
FRACTIONS = AnalysisConfig.FRACTIONS

print(f"Target Graph: {GRAPH_PATH}")
print(f"Random Simulations: {NUM_SIMULATIONS}")

In [ ]:
if not Path(GRAPH_PATH).exists():
    raise FileNotFoundError(f"{GRAPH_PATH} not found.")

print(f"Loading {GRAPH_PATH}...")
with open(GRAPH_PATH, 'rb') as f:
    G = pickle.load(f)
    
analyzer = NetworkAnalyzer(G)
storage = ResultsManager(metrics_dir=project_root / "metrics")
viz = NetworkVisualizer()

## 1. Global Metrics

In [ ]:
global_metrics = storage.get_cached_or_run(
    COUNTRY,
    "global_metrics",
    lambda: analyzer.calculate_global_metrics()
)
print(global_metrics)

In [ ]:
# Degree Distribution
viz.plot_degree_distribution(G, bins=50, title=f"Degree Distribution - {COUNTRY.title()}")

## 2. Robustness: Efficiency Decay
Comparing Random Failures vs Targeted Attacks (Degree & Betweenness).

In [ ]:
# 2.1 Random (Efficiency)
params_random = {'num_simulations': NUM_SIMULATIONS, 'fractions': FRACTIONS}
random_eff = storage.get_cached_or_run(
    COUNTRY,
    "efficiency_decay_random",
    lambda: analyzer.simulate_random_attacks(FRACTIONS, NUM_SIMULATIONS),
    current_params=params_random
)

# 2.2 Targeted Degree
targeted_deg = storage.get_cached_or_run(
    COUNTRY,
    "efficiency_decay_degree",
    lambda: analyzer.simulate_targeted_attack(FRACTIONS, strategy_name='degree')
)

# 2.3 Targeted Betweenness 
# (Note: betweenness calc on large graphs is slow, smart caching helps here!)
targeted_bet = storage.get_cached_or_run(
    COUNTRY,
    "efficiency_decay_betweenness",
    lambda: analyzer.simulate_targeted_attack(FRACTIONS, strategy_name='betweenness')
)

# 2.4 Targeted Inverse Degree
targeted_inv_deg = storage.get_cached_or_run(
    COUNTRY,
    "efficiency_decay_inverse_degree",
    lambda: analyzer.simulate_targeted_attack(FRACTIONS, strategy_name='inverse_degree')
)

# 2.5 Targeted Inverse Betweenness
targeted_inv_bet = storage.get_cached_or_run(
    COUNTRY,
    "efficiency_decay_inverse_betweenness",
    lambda: analyzer.simulate_targeted_attack(FRACTIONS, strategy_name='inverse_betweenness')
)

# 2.6 Targeted Articulation (LCC Minimization)
targeted_art = storage.get_cached_or_run(
    COUNTRY,
    "efficiency_decay_articulation",
    lambda: analyzer.simulate_targeted_attack(FRACTIONS, strategy_name='articulation')
)

In [ ]:
# Visualization
efficiency_decay_widget = viz.plot_efficiency_decay({
    'Random Failure': random_eff['efficiency'],
    'Targeted (Degree)': targeted_deg,
    'Targeted (Betweenness)': targeted_bet,
    'Inverse Targeted (Degree)': targeted_inv_deg,
    'Inverse Targeted (Betweenness)': targeted_inv_bet,
    'Targeted (Articulation)': targeted_art
}, title=f"Network Efficiency Decay - {COUNTRY.title()}")
display(efficiency_decay_widget)

In [ ]:
# 3. Robustness: Extended Metrics (All Strategies)
# Calculating Average Degree, Clustering, Diameter, Path Length for ALL strategies.
# Random simulations might be time-consuming due to Diameter/Path Length calculations.

print("Running Extended Metrics Analysis for ALL Strategies...")
full_metrics = ['average_degree', 'clustering', 'diameter', 'avg_path_length', 'efficiency', 'lcc']

if 'analyzer' in locals():
    # 1. Random
    print("  - Random (this may take time)...")
    results_random = storage.get_cached_or_run(
        COUNTRY,
        "extended_metrics_random",
        lambda: analyzer.simulate_random_attacks(FRACTIONS, NUM_SIMULATIONS, metrics=full_metrics)
    )

    # 2. Targeted Strategies
    target_strategies = [
        ('degree', 'Targeted (Degree)'),
        ('betweenness', 'Targeted (Betweenness)'),
        ('inverse_degree', 'Inverse Targeted (Degree)'),
        ('inverse_betweenness', 'Inverse Targeted (Betweenness)'),
        ('articulation', 'Targeted (Articulation)')
    ]

    results_targeted = {}
    for strat, label in target_strategies:
        print(f"  - {label}...")
        results_targeted[strat] = storage.get_cached_or_run(
            COUNTRY,
            f"extended_metrics_{strat}",
            lambda: analyzer.simulate_targeted_attack(FRACTIONS, strategy_name=strat, metrics=full_metrics)
        )


In [ ]:
# Plot Average Degree Decay
plot_data = { 'Random': results_random['average_degree'] }
for strat, label in target_strategies:
    if strat in results_targeted:
        plot_data[label] = results_targeted[strat]['average_degree']

avg_degree_widget = viz.plot_metric_decay(plot_data, title=f"Average Degree Decay - {COUNTRY.title()}", ylabel="Average Degree", log_x=True)
display(avg_degree_widget)


In [ ]:
# Plot Clustering Coefficient Decay
plot_data = { 'Random': results_random['clustering'] }
for strat, label in target_strategies:
    if strat in results_targeted:
        plot_data[label] = results_targeted[strat]['clustering']

clustering_widget = viz.plot_metric_decay(plot_data, title=f"Clustering Coefficient Decay - {COUNTRY.title()}", ylabel="Clustering Coefficient", log_x=True)
display(clustering_widget)


In [ ]:
# Plot Diameter Decay
plot_data = { 'Random': results_random['diameter'] }
for strat, label in target_strategies:
    if strat in results_targeted:
        plot_data[label] = results_targeted[strat]['diameter']

diameter_widget = viz.plot_metric_decay(plot_data, title=f"Diameter Decay - {COUNTRY.title()}", ylabel="Diameter (LCC)", log_x=True)
display(diameter_widget)


In [ ]:
# Plot Avg Path Length Decay
plot_data = { 'Random': results_random['avg_path_length'] }
for strat, label in target_strategies:
    if strat in results_targeted:
        plot_data[label] = results_targeted[strat]['avg_path_length']

avg_path_length_widget = viz.plot_metric_decay(plot_data, title=f"Avg Path Length Decay - {COUNTRY.title()}", ylabel="Avg Path Length (LCC)", log_x=True)
display(avg_path_length_widget)


In [ ]:
# Map visualization
viz.create_interactive_map_ui(G)